<a href="https://colab.research.google.com/github/Choi-DODO/SOC-Regional-Inequality/blob/main/Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Created on Fri Mar 24 14:45:11 2023

@author: user
"""

# import the pandas modules
import pandas as pd
import numpy as np
import os
import math
import re
from statsmodels.iolib.summary2 import summary_col
from collections import Counter

In [ ]:
from google.colab.data_table import DataTable
DataTable.max_columns = 40

from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = r'/content/drive/MyDrive/Acsblt2Prc/raw_data/panel_10_19_lf.csv'
panel = pd.read_csv(path, sep = ',', header = 0, ) #'euc-kr' for files in Korean
panel.head()

code_list = np.unique(panel['code'])
len(code_list)
panel_code = set(code_list)
panel.loc[panel['code'] == 29010, :]

,code,year,pop,area,company,worker,old_rate,old,econ_rate,econ,...,land4,ac1,ac2,ac3,ac4,ac5,ac6,ac7,ac8,ac9
740,29010.0,2010.0,81871.0,361.380,4943.0,27399.0,17.655,14454.0,67.153,54979.0,...,4.573,148.5,147.5,763902.1,674155.1,119697.5,102113.6,0.763,1.275,536226.9
741,29010.0,2011.0,82890.0,361.433,5268.0,29696.0,16.431,13620.0,68.697,56943.0,...,5.412,147.8,141.3,778460.1,700202.5,124864.1,103473.4,0.762,1.269,561700.3
742,29010.0,2012.0,113117.0,464.883,6640.0,46512.0,15.218,17214.0,68.425,77400.0,...,6.339,147.8,140.3,798375.3,707025.8,182237.6,142027.1,0.778,1.259,810126.5
743,29010.0,2013.0,122153.0,464.900,7468.0,56867.0,14.701,17958.0,68.793,84033.0,...,6.770,147.2,140.4,822235.9,717883.8,213830.9,167192.5,0.789,1.260,981207.8
744,29010.0,2014.0,156125.0,464.897,9124.0,70661.0,12.667,19777.0,69.107,107894.0,...,7.313,147.4,140.7,834680.2,722033.3,265483.2,211503.9,0.795,1.251,1177901.5
745,29010.0,2015.0,210884.0,464.866,10480.0,83764.0,10.622,22399.0,69.268,146074.0,...,7.151,146.9,139.4,856593.1,731440.5,315899.5,246504.5,0.812,1.227,1402953.3
746,29010.0,2016.0,243048.0,464.869,11853.0,90672.0,9.956,24197.0,69.642,169264.0,...,7.151,146.2,137.4,875663.4,744187.2,387075.4,305356.5,0.831,1.222,1702472.5
747,29010.0,2017.0,280100.0,464.893,13668.0,99827.0,9.586,26851.0,69.755,195384.0,...,8.221,146.0,139.5,826285.9,689438.6,346231.0,290262.0,0.866,1.244,1679510.3
748,29010.0,2018.0,314126.0,464.913,15871.0,115585.0,9.289,29178.0,70.076,220126.0,...,8.662,145.8,139.8,847057.5,695893.1,412632.0,349738.0,0.880,1.248,1916023.4
749,29010.0,2019.0,340575.0,464.950,18041.0,125410.0,9.381,31949.0,70.192,239056.0,...,8.662,145.4,139.6,865249.7,699334.9,452126.4,385926.6,0.889,1.247,2077744.1


In [ ]:
path_map = r'/content/drive/MyDrive/Acsblt2Prc/raw_data/district_code.csv'
map = pd.read_csv(path_map, sep = ',', header = 0, ) #'euc-kr' for files in Korean
map.drop(columns=['b_code', 'gen_date'], inplace= True, errors='raise')

,dist_code,h_code,시군구
0,11.0,1100000000,서울특별시
1,11010.0,1111000000,종로구
2,1101072.0,1111051500,청운효자동
3,1101072.0,1111051500,청운효자동
4,1101072.0,1111051500,청운효자동


In [ ]:
db_10 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2010.csv', encoding='UTF8'); db_10['year'] = 2010; price_10 = db_10.groupby(['시군구'], as_index=False).mean() ; del db_10
db_12 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2012.csv', encoding='cp949'); db_12['year'] = 2012; price_12 = db_12.groupby(['시군구'], as_index=False).mean() ; del db_12
db_13 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2013.csv', encoding='UTF8'); db_13['year'] = 2013; price_13 = db_13.groupby(['시군구'], as_index=False).mean() ; del db_13
db_14 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2014.csv', encoding='UTF8'); db_14['year'] = 2014; price_14 = db_14.groupby(['시군구'], as_index=False).mean() ; del db_14
db_15 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2015.csv', encoding='UTF8'); db_15['year'] = 2015; price_15 = db_15.groupby(['시군구'], as_index=False).mean() ; del db_15
db_16 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2016.csv', encoding='cp949'); db_16['year'] = 2016; price_16 = db_16.groupby(['시군구'], as_index=False).mean() ; del db_16
db_17 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2017.csv', encoding='cp949'); db_17['year'] = 2017; price_17 = db_17.groupby(['시군구'], as_index=False).mean() ; del db_17
db_18 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2018.csv', encoding='cp949'); db_18['year'] = 2018; price_18 = db_18.groupby(['시군구'], as_index=False).mean() ; del db_18
db_19 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2019.csv', encoding='cp949'); db_19['year'] = 2019; price_19 = db_19.groupby(['시군구'], as_index=False).mean() ; del db_19

In [ ]:
#@title Same name, Different areas
# 2011년 행정동 코드 부재: 남구, 동구 등 동명이역 일일이 매핑

db_11 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2011.csv', encoding='cp949'); db_11['year'] = 2011;

bins = list(db_11[db_11['일련번호'] == 1].index) #시군구 시작지점 기준 bin 카테고리 만들기
db_11.reset_index(inplace = True) #index열 생성
bins = bins + [9999999999999] #끝지점 지정
rc = pd.cut(db_11['index'], bins, right = False, labels = range(0,251, 1)) #시군구별 임시 식별 코드 부여
db_11['rc'] = rc.astype('int');

price_11 = db_11.groupby(['시군구', 'rc'], as_index=False).mean()

du_name = price_11.loc[price_11.duplicated(subset = ['시군구']), '시군구'].unique() #중복된 시군구명 정리
du_price_11 = price_11.loc[price_11['시군구'].isin(du_name),:].copy() #중복 시군구만 따로 정리
price_11 = price_11.loc[~(price_11['시군구'].isin(du_name)),:].copy() #비중복 시군구만 따로 정리

#인덱스 초기화
du_price_11.reset_index(drop = True, inplace = True)
price_11.reset_index(drop = True, inplace = True)

#i = 6
#a = db_11.loc[db_11['시군구'] == du_name[i], 'rc'].unique();
#b = db_11.loc[db_11['rc'].isin(a),'시도'].unique()
#print(a); print(b); print(du_name[i])

altered_du = {15:1150000000, 36:2644000000, 134 :4282000000, 242:4882000000, 31:2629000000, 44:2720000000, 51:2817700000, 61:2915500000, 70:3114000000,
              27:2617000000, 42:2714000000, 50:2814000000, 59:2911000000, 64:3011000000, 71:3117000000,
              32:2632000000, 45:2723000000, 62:2917000000, 72:3120000000,
              26:2614000000, 43:2717000000, 56:2826000000, 60:2914000000, 66:3017000000,1:1114000000,
              25:2611000000, 41:2711000000, 49:2811000000, 65:3014000000, 69:3111000000}

#동명이역 행정동 코드 부여
for rc in altered_du.keys():
  du_price_11.loc[du_price_11['rc'] == rc,'h_code'] = altered_du[rc]

#행정구역 재부여 및 데이터 디멘션 통일(시군구 drop)
du_price_11 = pd.merge(du_price_11, map, how='left', on=['h_code']) # 시도단위 -> 주택유형 (raw 분류)
du_price_11.drop(columns=['index', '시군구_x', '시군구_y', 'rc', '일련번호'], inplace= True, errors='raise')
du_price_11.columns = ['area', 'price', 'year','h_code', 'dist_code']
mduprice_11 = du_price_11.groupby(['dist_code'], as_index=False).mean()

<ipython-input-49-b9a90b9b9415>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  price_11 = db_11.groupby(['시군구', 'rc'], as_index=False).mean()


In [ ]:
#@title For data of Cheongju-Si (administrative area integration issue)
#2011년 행정동 코드 부재: 통합 청주시 따로 매핑

db_11['소재지'] = db_11['소재지'].str.replace(pat=r'[^ㄱ-ㅣ가-힣]+', repl= r'', regex=True) #소재지 숫자 버리기
names = db_11.loc[(db_11['시군구'].str.contains('청주', na = False)), '소재지'].unique()   #새로 배정해야할 소재지들

db_11.loc[(db_11['시군구'].str.contains('청주', na = False)) & (db_11['소재지'].str.contains('영동', na = False)), :]


#43111
key1 = ['영동', '북문로가동', '남문로가동', '문화동', '서운동', '서문동', '남주동', '석교동', '수동', '수동산', '탑동',
         '대성동', '영운동', '금천동', '용담동', '용담동산', '명암동', '명암동산', '산성동', '산성동산', '용암동', '용암동산',
         '용정동', '용정동산', '방서동', '평촌동', '지북동', '지북동산', '운동동', '운동동산', '월오동', '월오동산']
val1 = [4311100000 for i in range(len(key1))]

#43112
key2 = ['사직동', '사직동산', '사창동', '사창동산', '모충동','산남동', '산남동산', '미평동', '미평동산', '분평동', '수곡동', '수곡동산',
        '성화동', '성화동산', '성화동가', '개신동', '개신동산', '죽림동', '죽림동산', '장성동', '장성동산', '장암동', '장암동산']
val2 = [4311200000 for i in range(len(key2))]

#43113
key3 = ['운천동', '운천동산', '신봉동', '신봉동산', '가경동', '가경동산', '복대동', '복대동산', '봉명동', '봉명동산', '송정동', '강서동', '강서동산',
        '석곡동', '석곡동산', '휴암동', '휴암동산', '신전동', '신전동산', '현암동', '현암동산', '동막동',
        '동막동산', '수의동', '수의동산', '지동동', '지동동산', '서촌동', '서촌동산', '신성동', '신성동산',
        '평동', '평동산', '신대동', '남촌동', '남촌동산', '내곡동', '내곡동산', '상신동', '상신동산',
        '원평동', '원평동산', '문암동', '송절동', '송절동산', '화계동', '외북동', '외북동산', '향정동',
        '비하동', '비하동산', '석소동', '석소동산', '정봉동', '정봉동산', '신촌동']
val3 = [4311300000 for i in range(len(key3))]

#43114
key4 = ['주성동', '주성동산', '주성동가', '주중동', '주중동산', '주중동가', '정상동', '정상동산', '정하동', '정하동산', '정북동', '오동동', '오동동산',
        '외남동', '외남동산', '외평동', '외하동', '우암동', '우암동산', '내덕동', '내덕동산', '율량동', '율량동산', '율량동가', '사천동', '사천동산' ]
val4 = [4311400000 for i in range(len(key4))]

dict1 = dict(zip(key1, val1)); dict2 = dict(zip(key2, val2)); dict3 = dict(zip(key3, val3)); dict4 = dict(zip(key4, val4))

# 행정동 코드 채워넣기
for dicts in [dict1, dict2, dict3, dict4]:
  for i in range(len(dicts.keys())):
    db_11.loc[(db_11['시군구'].str.contains('청주', na = False)) & (db_11['소재지'].str.contains(list(dicts.keys())[i], na = False)), 'h_code'] = list(dicts.values())[i]
db_11.loc[(db_11['시군구'].str.contains('청주', na = False)) & (db_11['h_code'].isna()), 'h_code'] = 4311100000 # 이런 동이름은 빠졌음 남문로2가동 76-5

#청주시 데이터만 따로 분류
cj_11 = db_11.loc[(db_11['시군구'].str.contains('청주', na = False)), :].copy()

cj_11 = pd.merge(cj_11, map, how='left', on='h_code') # 행정동 코드 -> 행정구역 코드
cj_11.loc[cj_11['dist_code'].isna(),:] #데이터 무결성 확인.

#행정구역 코드 단위 평균 지가 도출
mcj_11 = cj_11.groupby(['dist_code'], as_index=False).mean()

#concat 위해 컬럼명 및 디멘션 통일
mcj_11.columns
mcj_11.drop(columns=['일련번호', 'index', 'rc'], inplace= True, errors='raise')
mcj_11.columns = ['dist_code', 'area', 'price', 'year', 'h_code']

<ipython-input-50-ad8157fed2ad>:49: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mcj_11 = cj_11.groupby(['dist_code'], as_index=False).mean()


In [ ]:
#@title For data without region code in year of 2011
#2011년 행정동 코드 부재: 결측지 따로 매핑

#당진군 -> 당진시, 부천 비자치구 통합, 여주군 -> 여주시 승격. 연기군 -> 세종. 청원군 -> 청주시 청원구
#코드가 아닌 지역명 밖에 없는 데이터. 지역명 단위로 merge하고 코드 없는 지역 확인.

#청주/청원군 자료 제외 + 매핑위해 중복되는 h_code 삭제
price_11 = price_11.loc[~(price_11['시군구'].str.contains('청주', na = False)), :]
price_11 = price_11.loc[~(price_11['시군구'].str.contains('청원', na = False)), :]


price_11 = pd.merge(price_11, map, how='left', on=['시군구']) # 시도단위 -> 주택유형 (raw 분류)
name = price_11.loc[price_11['h_code'].isna(),:'시군구'] #행정구역-행정동 mapping 후 결측값 확인

#결측값 법정동 코드 사전 작성.
key_list_11 = name['시군구'].tolist()
code_list_11 = [4128100000, 4128500000, 4128700000, 4427000000, 4119000000, 4119000000, 4119000000, 4113500000, 4113100000, 4113300000,
 4111300000, 4111700000, 4111100000, 4111500000, 4127300000, 4127100000, 4117300000, 4117100000, 4167000000, 3611000000,
 4146300000, 4146500000, 4146100000, 4511300000, 4511100000, 4812500000, 4812700000, 4812300000, 4812100000, 4812900000,
 4413100000, 4413300000, 4711100000, 4711300000]
altered_gu_11 = dict(zip(key_list_11, code_list_11))

#결측값 행정동 코드 부여
for gu in altered_gu_11.keys():
  price_11.loc[price_11['시군구'] == gu,'h_code'] = altered_gu_11[gu]

#행정구역 재부여를 위한 drop 및 데이터 컬럼명 정제
price_11.drop(columns=['시군구', 'rc', 'index', '일련번호', 'dist_code'], inplace= True, errors='raise')
price_11.columns = ['area', 'price', 'year','h_code']

#행정구역 재부여 및 데이터 디멘션 통일(시군구 drop)
price_11 = pd.merge(price_11, map, how='left', on=['h_code']) # 시도단위 -> 주택유형 (raw 분류)
price_11.drop(columns=['시군구'], inplace= True, errors='raise')
price_11.head()

#행정구역 코드 단위 평균 지가 도출
mprice_11 = price_11.groupby(['dist_code'], as_index=False).mean()
mprice_11 = pd.concat([mprice_11, mduprice_11, mcj_11]) # 동명이역, 청주, 나머지 2011년 데이터 통일.

In [ ]:
#@title Matching data demansion and column names before concat

price_10.columns
price_10.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_10.columns = ['h_code', 'area', 'price', 'year']

price_12.columns
price_12.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_12.columns = ['h_code', 'area', 'price', 'year']

price_13.columns
price_13.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_13.columns = ['h_code', 'area', 'price', 'year']

price_14.columns
price_14.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_14.columns = ['h_code', 'area', 'price', 'year']

price_15.columns
price_15.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_15.columns = ['h_code', 'area', 'price', 'year']

price_16.columns
price_16.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_16.columns = ['h_code', 'area', 'price', 'year']

price_17.columns
price_17.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_17.columns = ['h_code', 'area', 'price', 'year']

price_18.columns
price_18.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_18.columns = ['h_code', 'area', 'price', 'year']

price_19.columns
price_19.drop(columns=['일련번호', '읍면동리', '지번구분'], inplace= True, errors='raise')
price_19.columns = ['h_code', 'area', 'price', 'year']

In [ ]:
#@title 마창진, 통합청주시, 2011년 제외 통합 데이터

#과거 행정동 코드 기반 현재 dist_code mapping
#인천 남구 -> 미추홀구, 부천시, 여주시, 청원군 -> 청주시 청원구, 연기군 -> 세종, 당진군 -> 당진시
#2817000000:23090, 4119500000:31050, 4119700000:31050, 4119900000:31050, 4173000000:31280, 4371000000:33044, 4473000000:29010, 4483000000:34080

#데이터 병합 이후 행정구역코드 mapping위해 h_code scale 조정
price_total = pd.concat([price_10, price_12, price_13, price_14, price_15, price_16, price_17, price_18, price_19]) # row bind : axis = 0, default
price_total['h_code'] = price_total['h_code'].mul(100000)

# mapping: 행정동 코드 -> 행정구역 코드
price_total = pd.merge(price_total, map, how='left', on='h_code')
name = price_total.loc[price_total['dist_code'].isna(),:] #행정구역-행정동 mapping 후 결측값 확인

key_list_total = list(np.unique(name['h_code']))
code_list_total = [23090, 31050, 31050, 31050, 31280, 33044, 29010, 34080]
altered_gu_total = dict(zip(key_list_total, code_list_total))


#창원, 마산, 진해는 2010년 통합 후 비자치구 단위로 코드가 부여됨. 따라서 해당 지역은 2010년에 대하여 비자치구 별 평균 지가를 계산 후 최종 데이터에 concat 할 것. 나머지 지역에 대하여서는 통합/변경 후에도 일관성이 유지되므로 그대로 mapping 실시
price_total.drop(price_total[price_total['h_code'].isin([4811000000, 4816000000, 4819000000])].index, inplace=True)  #마산시, 창원시, 진해시 데이터 제거

#청주시 소속 상당구, 서원구, 청원구, 흥덕구 4개 지역은 통합청주시가 2014년부터 출범됨에 따라서 2010 ~ 2014 기간은 따로 계산해야함. 따라서 해당 기간동안의 데이터 삭제
price_total.drop(price_total[(price_total['h_code'].isin([4311100000, 4317000000, 4311300000, 4371000000]))&(price_total['year'].isin([2010, 2012, 2013, 2014]))].index, inplace=True)  #마산시, 창원시, 진해시 데이터 제거

#결측값 행정구역 코드 부여
for gu in altered_gu_total.keys():
  price_total.loc[price_total['h_code'] == gu,'dist_code'] = altered_gu_total[gu]

#데이터 디멘션 통일
price_total.drop(columns=['시군구'], inplace= True, errors='raise')
price_total.head()

#행정구역 코드 단위 평균 지가 도출
mprice_total = price_total.groupby(['dist_code', 'year'], as_index=False).mean()

In [ ]:
#@title 통합청주시 및 세종시 데이터
db_10 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2010.csv', encoding='UTF8'); db_10['year'] = 2010
db_12 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2012.csv', encoding='cp949'); db_12['year'] = 2012
db_13 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2013.csv', encoding='UTF8'); db_13['year'] = 2013
db_14 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2014.csv', encoding='UTF8'); db_14['year'] = 2014
cj_map = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/cj_map.csv', encoding='UTF8')
cj_map.rename(columns = {'전시구' : '시군구', '전동':'읍면동리', '후시구':'새시군구'}, inplace = True) #매핑 위해 컬럼명 통일

cj_10 = db_10.loc[db_10['시군구'].isin([43111, 43170, 43113, 43710]),:]; del db_10  #청주시 데이터 추출
cj_12 = db_12.loc[db_12['시군구'].isin([43111, 43170, 43113, 43710]),:]; del db_12  #청주시 데이터 추출
cj_13 = db_13.loc[db_13['시군구'].isin([43111, 43170, 43113, 43710]),:]; del db_13  #청주시 데이터 추출
cj_14 = db_14.loc[db_14['시군구'].isin([43111, 43170, 43113, 43710]),:]; del db_14  #청주시 데이터 추출

cj_1014 = pd.concat([cj_10, cj_12, cj_13, cj_14]) #청주시 데이터 10~14 결합

cj_new = pd.merge(cj_1014, cj_map, how='left', on=['시군구', '읍면동리']) # 새로운 행정구역(새시군구) 코드 부여
cj_new.loc[cj_new['새시군구'].isna(),'새시군구'] = 36110 #세종시 편입된 동네 (부용면) 식별

#평균지가 도출
mcj_1014 = cj_new.groupby(['새시군구', 'year'], as_index=False).mean()

#데이터 디멘션 통일
mcj_1014.drop(columns=['일련번호', '시군구', '읍면동리', '지번구분'], inplace= True, errors='raise')
mcj_1014.head()
mcj_1014.columns = ['h_code', 'year', 'area', 'price']
mcj_1014['h_code'] = mcj_1014['h_code'].mul(100000)   # mapping위해 rescaling
mcj_1014 = pd.merge(mcj_1014, map, how='left', on=['h_code']) # mapping 새로운 행정동 기준으로 행정구역 코드 매핑
mcj_1014.drop(columns=['시군구'], inplace= True, errors='raise')

<ipython-input-54-669f67228fb0>:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  db_10 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2010.csv', encoding='UTF8'); db_10['year'] = 2010
<ipython-input-54-669f67228fb0>:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  db_12 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2012.csv', encoding='cp949'); db_12['year'] = 2012
<ipython-input-54-669f67228fb0>:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  db_13 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2013.csv', encoding='UTF8'); db_13['year'] = 2013
<ipython-input-54-669f67228fb0>:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  db_14 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2014.csv', encoding='UTF8'); db_14['y

In [ ]:
#@title 마창진 데이터
#마산, 창원, 진해, 2010년 비자치구 단위 평균값 도출.
#2010년 창원 데이터만 추출
db_10 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2010.csv', encoding='UTF8'); db_10['year'] = 2010;
cw_10 = db_10.loc[db_10['시군구'].isin([48110, 48160, 48190]),:]; del db_10  #창원, 마산, 진해 순


cw_10['before'] = cw_10['시군구'].mul(100000) + cw_10['읍면동리'] #읍면동 단위 code 재설정
cw_map = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/cw_map.csv', encoding='UTF8') #통합창원시 전후 mapping 자료
cw_10 = pd.merge(cw_10, cw_map, how='left', on=['before']) # mapping 통합창원시 기준 비자치구 행정동 코드

cw_10.loc[cw_10['after'].isna(),:] #폐지, 통합된 지역 발견 4811011000, 4811011100, 4811011300

#결측값 행정구역 코드 부여 상남동 등 -> 성산구 편입
altered_cw = {4811011000:4812353000, 4811011100:4812353000, 4811011300:4812352000}
for dong in altered_cw.keys():
  cw_10.loc[cw_10['before'] == dong,'after'] = altered_cw[dong]

#시군구 단위 행정구역 코드만 추출
cw_10['h_code'] = cw_10['after'].div(100000)
for i in range(len(cw_10)):
  cw_10.loc[i,'h_code'] = math.floor(cw_10.loc[i,'h_code'])


cw_10['h_code'] = cw_10['h_code'].mul(100000)  #mapping 위해 scale 조정

cw_10 = pd.merge(cw_10, map, how='left', on='h_code') # 행정동 코드 -> 행정구역 코드
cw_10.loc[cw_10['dist_code'].isna(),:]

#행정구역 코드 단위 평균 지가 도출
mcw_10 = cw_10.groupby(['dist_code'], as_index=False).mean()

#concat 위해 컬럼명 및 디멘션 통일
mcw_10.columns
mcw_10.drop(columns=['일련번호', '시군구_x', '읍면동리', '지번구분', 'before', 'after'], inplace= True, errors='raise')
mcw_10.columns = ['dist_code', 'area', 'price', 'year', 'h_code']

<ipython-input-55-463c27a1f3ca>:4: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  db_10 = pd.read_csv('/content/drive/MyDrive/Acsblt2Prc/raw_data/2010.csv', encoding='UTF8'); db_10['year'] = 2010;
<ipython-input-55-463c27a1f3ca>:31: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mcw_10 = cw_10.groupby(['dist_code'], as_index=False).mean()


In [ ]:
#@title 전체 데이터(11년 제외) + 창원데이터 + 청주데이터 + 11년

land_price = pd.concat([mprice_11, mprice_total, mcj_1014, mcw_10])
land_price.drop(columns=['h_code'], inplace= True, errors='raise')
land_price.rename(columns = {'dist_code' : 'code'}, inplace = True) #패널 데이터와 key 컬럼명 통일
land_price = land_price.groupby(['code', 'year'], as_index=False).mean()
land_price.sort_values(by = ['year', 'code'], inplace=True)
land_price.reset_index(inplace = True, drop = True) #index열 초기화

land_price.to_excel("/content/drive/MyDrive/Acsblt2Prc/land_price.xlsx",
                     sheet_name = 'Sheet1',
                     na_rep = 'NaN',
                     float_format = "%.2f",
                     header = True,
                     index = False,
                     #index_label = "id",
                     startrow = 0,
                     startcol = 0,
                     #engine = 'xlsxwriter',
                     freeze_panes = (1, 1))

,code,year,area,price
0,11010.0,2010.0,376.155262,6.052190e+06
1,11020.0,2010.0,416.231802,7.662376e+06
2,11030.0,2010.0,533.811825,4.581932e+06
3,11040.0,2010.0,441.006641,3.296481e+06
4,11050.0,2010.0,1011.553987,3.072775e+06
...,...,...,...,...
2495,38380.0,2019.0,4741.772575,4.686374e+04
2496,38390.0,2019.0,5199.089456,7.545907e+04
2497,38400.0,2019.0,5470.266103,2.955091e+04
2498,39010.0,2019.0,3940.274276,3.239430e+05


In [ ]:
#@title Cross-check of data validity
# 패널 데이터와 코드 비교

#패널 코드 집합
code_list = np.unique(panel['code'])
len(code_list)
panel_code = set(code_list)

#지가자료 코드 집합
code_list = np.unique(land_price['code'])
len(code_list)
price_code = set(code_list)

panel_code - price_code  #지가자료에 없는 데이터 = 26050-울주군
price_code - panel_code  #패널자료에 없는 데이터 = 26310-울주군, 37430-울릉군, 39010-제주시, 39020-서귀포시


#패널 데이터를 지가 데이터에 맞추기 (울주군 코드 변경)
#land_price.loc[land_price['code'] == 26310, 'code'] = 26050
panel.loc[panel['code'] == 26050, 'code'] = 26310

#psize = pd.DataFrame(panel.groupby(['code']).size()) ;# 247개 지역 10개년치
#lsize = pd.DataFrame(land_price.groupby(['code']).size()) ; lsize.loc[lsize.iloc[:,0] != 10,:] #250개 지역 10개년치 missing 존재
#land_price.loc[land_price['code'].isin([33042]),:] #33042 (청주 서원구) 5개 (2010 ~ 2014 missing) 존재
# 2018년 남구 -> 미추홀구, 2014년 통합청주시 출범

In [ ]:
#@title Merge land price + original panel data + accessibility data
#지가데이터 + 패널데이터 + 접근도 데이터

total_data = pd.merge(panel, land_price, how='left', on=['code', 'year']) #시군구 -> 연도순으로 매칭
#중복된 code 한번 더 평균
total_data = total_data.groupby(['code', 'year'], as_index=False).mean()
#필지 면적 컬럼명 변경
total_data.columns
total_data.columns = ['code', 'year', 'pop', 'area', 'company', 'worker', 'old_rate', 'old',
                      'econ_rate', 'econ', 'grdp', 'restrict', 'house1', 'house2', 'manu1','manu2',
                      'rp', 'tax_t', 'tax_p', 'land1', 'land2', 'land3', 'land4',
                      'ac1', 'ac2', 'ac3', 'ac4', 'ac5', 'ac6', 'ac7', 'ac8', 'ac9', 'land_area', 'land_price', 'mland_price']

acs = pd.read_csv(r'/content/drive/MyDrive/Acsblt2Prc/raw_data/acsblt.csv', sep = ',', header = 0, ) #'euc-kr' for files in Korean
total_data = pd.merge(total_data, acs, how='left', on=['code', 'year']) #시군구 -> 연도순으로 매칭

total_data[total_data['id'].isna()]

,code,year,pop,area,company,worker,old_rate,old,econ_rate,econ,...,ac7,ac8,ac9,land_area,land_price,mland_price,zone,road,train,id


In [ ]:
#@title SIG_code mapping

sig = pd.read_csv(r'/content/drive/MyDrive/Acsblt2Prc/raw_data/SIG_CD.csv', sep = ',', header = 0, ) #'euc-kr' for files in Korean
total_data = pd.merge(total_data, sig, how='left', left_on='code', right_on = 'dist_code') #시군구 -> 연도순으로 매칭
total_data.drop(columns=['dist_code'], inplace= True, errors='raise')
total_data.head()

,code,year,pop,area,company,worker,old_rate,old,econ_rate,econ,...,ac8,ac9,land_area,land_price,mland_price,zone,road,train,id,SIG_CD
0,11010.0,2010.0,170578.0,23.907,37636.0,238510.0,14.685,25050.0,73.485,125350.0,...,1.390,5382409.3,376.155262,6.052190e+06,16089.606974,1,19861.468,21656.448,1,11110.0
1,11010.0,2011.0,168382.0,23.907,38635.0,232685.0,13.541,22800.0,75.041,126355.0,...,1.358,5449139.9,382.995959,6.088357e+06,15896.660212,1,19861.468,21977.904,1,11110.0
2,11010.0,2012.0,165207.0,23.907,39486.0,232857.0,14.364,23730.0,74.674,123367.0,...,1.317,5345762.6,376.765524,6.292539e+06,16701.473411,1,19862.136,22028.072,1,11110.0
3,11010.0,2013.0,160070.0,23.907,39326.0,235681.0,14.902,23853.0,74.487,119232.0,...,1.319,5415727.1,380.477948,6.375410e+06,16756.318689,1,19887.216,22028.896,1,11110.0
4,11010.0,2014.0,156993.0,23.913,40923.0,245698.0,15.527,24376.0,74.312,116664.0,...,1.316,5428940.3,390.879385,6.395202e+06,16361.061213,1,19933.124,22028.76,1,11110.0


In [ ]:
#@title Save final output
total_data.to_excel("/content/drive/MyDrive/Acsblt2Prc/final_data/panel_land.xlsx",
                     sheet_name = 'Sheet1',
                     na_rep = 'NaN',
                     float_format = "%.2f",
                     header = True,
                     index = False,
                     #index_label = "id",
                     startrow = 0,
                     startcol = 0,
                     #engine = 'xlsxwriter',
                     freeze_panes = (1, 1))

In [ ]:
#@title 최종 데이터 불러오기

"""total_data = pd.read_excel("/content/drive/MyDrive/Acsblt2Prc/final_data/panel_land.xlsx",
                           sheet_name = 'Sheet1',
                           #index_col = 'id',
                           header = 0)


df10 = total_data.loc[total_data['year'] == 2010,:]; df15 = total_data.loc[total_data['year'] == 2015,:]
df11 = total_data.loc[total_data['year'] == 2011,:]; df16 = total_data.loc[total_data['year'] == 2016,:]
df12 = total_data.loc[total_data['year'] == 2012,:]; df17 = total_data.loc[total_data['year'] == 2017,:]
df13 = total_data.loc[total_data['year'] == 2013,:]; df18 = total_data.loc[total_data['year'] == 2018,:]
df14 = total_data.loc[total_data['year'] == 2014,:]; df19 = total_data.loc[total_data['year'] == 2019,:]

df19.to_excel("/content/drive/MyDrive/Acsblt2Prc/final_data/panel_19.xlsx",
              sheet_name = 'Sheet1',
              na_rep = 'NaN',
              float_format = "%.2f",
              header = True,
              index = False,
              #index_label = "id",
              startrow = 0,
              startcol = 0,
              #engine = 'xlsxwriter',
              freeze_panes = (1, 1))"""